In [4]:
import pandas as pd
import numpy as np
import urllib2
from bs4 import BeautifulSoup
from time import sleep
import requests
import json

# Misc Functions

In [60]:
def bleach(string):
    temp = ""
    string = str(string)
    for i in string:
        if i in ["1","2","3","4","5","6","7","8","9","0","."]:
            temp += i
    if len(temp) > 0:
        return float(temp)

def combine_list(list):
    temp = ""
    for i in list:
        temp += " " + i
    return temp

# Get API info from IMDBid

In [7]:

## gets complete information from omdbapi using the imdb id

def get_api_from_id(title_id):
    this_url = "http://www.omdbapi.com/?i=" + title_id + "&plot=full&r=json&apikey=9f5296af"
    req = requests.get(this_url)
    return req.json()
#     print req.json()
    sleep(2)

In [104]:
def df_from_api(title_id):
    df = pd.DataFrame(data=[title_id], columns=['imdb_id'])
    ## check the column names here against the column names in the dataset
    df['json'] = df['imdb_id'].apply(get_api_from_id)
    df['name'] = df['json'].apply(lambda x: x['Title'])
    df['genres'] = df['json'].apply(lambda x: str.lower(str(x['Genre'])))
    df['seasons'] = df['json'].apply(lambda x: bleach(x['totalSeasons']))
    df['runtime'] = df['json'].apply(lambda x: bleach(x['Runtime']))
    df['released'] = df['json'].apply(lambda x: x['Released'])
    sleep(2)
    return df

# Scrape Network

In [66]:
def scrape_network(id):
    words = ""
    url = "http://www.imdb.com/title/" + id + "/companycredits?ref_=ttspec_sa_5"
    soup = BeautifulSoup(urllib2.urlopen(url))
    simpleLists = soup.find_all('ul', {'class': 'simpleList'})
    try:
        for li in simpleLists[1]('li'):
            for a in li('a'):
                words += (a.get_text() + '\n')
        return words.split("\n")[0]
#         print words.split("\n")[0]
    except:
        return "unknown"
    sleep(2)

# Scrape Keywords

In [67]:
## Accesses page based on imdb id

def access_keyword_page(imdbID):
    ## imdbID needs to be added in str format
    return 'http://www.imdb.com/title/' + imdbID + '/keywords?ref_=tt_stry_kw'

In [68]:
def scrape_keywords(imdbID):
    soup_for_keywords = BeautifulSoup(urllib2.urlopen(access_keyword_page(imdbID)))
    temp_keywords = []
    for div in soup_for_keywords('div', {'id':'keywords_content'}):
        for text in div('div', {'class':'sodatext'}):
            for a in text('a'):
    #             print a.get_text()
                temp_keywords.append(a.get_text())
    return temp_keywords
#     print temp_keywords
    sleep(2)

# Putting it all together

In [107]:
def get_tv_prediction(imdb_id):
    df1 = df_from_api(imdb_id)
    df1['keywords'] = df1['imdb_id'].apply(lambda x: combine_list(scrape_keywords(x)))
    df1['network'] = df1['imdb_id'].apply(lambda x: str.lower(str(scrape_network(x))))
    return df1

In [108]:
test_df = get_tv_prediction('tt0303461')
test_df

,imdb_id,json,name,genres,seasons,runtime,released,keywords,network
0,tt0303461,{u'Plot': u'Captain Malcolm 'Mal' Reynolds is ...,Firefly,"adventure, drama, sci-fi",1.0,44.0,20 Sep 2002,future captain space psychic spacecraft preac...,fox film corporation
